In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pymysql
from sqlalchemy import create_engine
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [3]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/covid_19_final_project'
engine = create_engine(connection_string)

In [4]:
#data = pd.read_sql_query('SELECT * FROM vaccination_eu_eea AS vax LEFT JOIN new_daily_cases AS cases ON vax.YearWeekISO=cases.dateRep;', engine)
#data.head()

In [5]:
#data.to_csv('../datasets_Tableau_clean/daily_vax_and_cases.csv')

These are way to many entries (6M). We need a different approach to joining both datasets.

In [26]:
vaccination=pd.read_csv('../datasets_Tableau_clean/vaccination_EU_EEA_clean.csv')
new_cases=pd.read_csv('../datasets_Tableau_clean/new_daily_cases_clean.csv')

vaccination['YearWeekISO']=pd.to_datetime(vaccination['YearWeekISO'],errors='coerce')
new_cases['dateRep'] = pd.to_datetime(new_cases['dateRep'],errors='coerce')

In [27]:
vaccination.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239853 entries, 0 to 239852
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Unnamed: 0        239853 non-null  int64         
 1   YearWeekISO       239853 non-null  datetime64[ns]
 2   ReportingCountry  239853 non-null  object        
 3   FirstDose         239853 non-null  int64         
 4   SecondDose        239853 non-null  int64         
 5   DoseAdditional1   239853 non-null  int64         
 6   UnknownDose       239853 non-null  int64         
 7   Region            239853 non-null  object        
 8   TargetGroup       239853 non-null  object        
 9   Vaccine           239853 non-null  object        
 10  Population        239853 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(4)
memory usage: 20.1+ MB


In [28]:
to_drop=['Unnamed: 0','Region']
vaccination_clean=vaccination.sort_values('YearWeekISO', axis=0, ascending=True).drop(to_drop,axis=1)

In [29]:
new_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22069 entries, 0 to 22068
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Unnamed: 0               22069 non-null  int64         
 1   dateRep                  22069 non-null  datetime64[ns]
 2   day                      22069 non-null  int64         
 3   month                    22069 non-null  int64         
 4   year                     22069 non-null  int64         
 5   cases                    22069 non-null  float64       
 6   deaths                   22069 non-null  float64       
 7   countriesAndTerritories  22069 non-null  object        
 8   geoId                    22069 non-null  object        
 9   countryterritoryCode     22069 non-null  object        
 10  popData2020              22069 non-null  int64         
 11  continentExp             22069 non-null  object        
dtypes: datetime64[ns](1), float64(2)

In [30]:
to_drop=['Unnamed: 0','day','month','year','countriesAndTerritories','countryterritoryCode','continentExp']
new_cases_clean=new_cases.sort_values(['dateRep', 'geoId'], axis=0, ascending=True).drop(to_drop,axis=1)
new_cases_clean_norm=new_cases_clean.copy()
new_cases_clean_norm['cases_per_100khab']=new_cases_clean_norm['cases']/new_cases_clean_norm['popData2020']*100000
new_cases_clean_norm['deaths_per_100khab']=new_cases_clean_norm['deaths']/new_cases_clean_norm['popData2020']*100000
new_cases_clean_norm=new_cases_clean_norm.drop(['cases','deaths','popData2020'], axis=1).reset_index().drop('index',axis=1)
new_cases_clean_norm['year_week']=new_cases_clean_norm['dateRep'].dt.strftime('%Y-%W')
new_cases_clean_norm=new_cases_clean_norm.drop(['dateRep'], axis=1)
new_cases_clean_norm
#normalize cases by population?

,geoId,cases_per_100khab,deaths_per_100khab,year_week
0,ES,0.002113,0.000000,2020-00
1,FI,0.018099,0.000000,2020-00
2,CZ,0.000000,0.000000,2020-00
3,DE,0.001202,0.000000,2020-00
4,ES,0.033803,0.000000,2020-00
...,...,...,...,...
22064,PT,188.774007,0.485630,2022-48
22065,RO,97.010488,0.812258,2022-48
22066,SE,27.276453,0.309850,2022-48
22067,SI,205.643409,0.715696,2022-48


In [31]:
vaccination_clean_grouped=vaccination_clean.groupby(['YearWeekISO','ReportingCountry'], as_index=False).agg({'FirstDose':'sum', 
                         'SecondDose':'sum', 
                         'Population':'mean'}).sort_values(['YearWeekISO', 'ReportingCountry'], axis=0, ascending=True)
vaccination_clean_grouped['first_dose_100khab']=vaccination_clean_grouped['FirstDose']/vaccination_clean_grouped['Population']*100000
vaccination_clean_grouped['second_dose_100khab']=vaccination_clean_grouped['SecondDose']/vaccination_clean_grouped['Population']*100000
vaccination_clean_grouped['year_week_vax']=vaccination_clean_grouped['YearWeekISO'].dt.strftime('%Y-%W')
vaccination_clean_grouped=vaccination_clean_grouped.drop(['YearWeekISO','FirstDose','SecondDose','Population'],axis=1)
vaccination_clean_grouped


,ReportingCountry,first_dose_100khab,second_dose_100khab,year_week_vax
0,DK,0.034348,0.000000,2020-50
1,DK,0.206088,0.000000,2020-51
2,BG,0.000000,0.000000,2020-52
3,CY,123.873176,0.000000,2020-52
4,CZ,34.037972,0.000000,2020-52
...,...,...,...,...
1830,NL,4.492295,7.077375,2022-09
1831,PL,100.663526,247.915216,2022-09
1832,PT,14.976822,31.799038,2022-09
1833,SE,27.092480,69.396642,2022-09


In [32]:
#Now I need to join/merge/concat these two "ready" dataframes to get the final dataset I will work with.

In [45]:
data=new_cases_clean_norm.merge(vaccination_clean_grouped,how='left',left_on=['year_week',
                                                      'geoId'],right_on=['year_week_vax','ReportingCountry'])

In [74]:
import datetime
data_ready=data.drop(['ReportingCountry','year_week_vax'],axis=1).fillna(0)
data_ready['year_week']=data_ready['year_week'].apply(lambda x: x.replace('-',''))+'6'
data_ready['date']=pd.to_datetime(data_ready['year_week'], format="%Y%W%w", errors='raise')
data_ready

#Don't use the date directly, but get info from it. Use the weekday, the week of the year, etc.
#Facebook's profit time series. Look up tutorials.

,geoId,cases_per_100khab,deaths_per_100khab,year_week,first_dose_100khab,second_dose_100khab,date
0,ES,0.002113,0.000000,2020006,0.0,0.0,2020-01-04
1,FI,0.018099,0.000000,2020006,0.0,0.0,2020-01-04
2,CZ,0.000000,0.000000,2020006,0.0,0.0,2020-01-04
3,DE,0.001202,0.000000,2020006,0.0,0.0,2020-01-04
4,ES,0.033803,0.000000,2020006,0.0,0.0,2020-01-04
...,...,...,...,...,...,...,...
22064,PT,188.774007,0.485630,2022486,0.0,0.0,2022-12-03
22065,RO,97.010488,0.812258,2022486,0.0,0.0,2022-12-03
22066,SE,27.276453,0.309850,2022486,0.0,0.0,2022-12-03
22067,SI,205.643409,0.715696,2022486,0.0,0.0,2022-12-03


In [75]:
data_ready[data_ready['geoId']=='ES'].head(20)

,geoId,cases_per_100khab,deaths_per_100khab,year_week,first_dose_100khab,second_dose_100khab,date
0,ES,0.002113,0.000000,2020006,0.0,0.0,2020-01-04
4,ES,0.033803,0.000000,2020006,0.0,0.0,2020-01-04
15,ES,1.445092,0.000000,2020006,0.0,0.0,2020-01-04
38,ES,11.499471,1.890874,2020006,0.0,0.0,2020-01-04
68,ES,1.527488,0.371837,2020006,0.0,0.0,2020-01-04
98,ES,0.836632,0.073945,2020016,0.0,0.0,2020-01-11
128,ES,1.174666,0.021127,2020016,0.0,0.0,2020-01-11
158,ES,7.612088,0.025352,2020016,0.0,0.0,2020-01-11
188,ES,24.133465,0.143664,2020016,0.0,0.0,2020-01-11
218,ES,22.303860,0.285216,2020016,0.0,0.0,2020-01-11


## Next Step: create accumulated data here, to work later with Tableau

In [66]:
data_ready.drop('year_week',axis=1,inplace=True)

In [67]:
new_columns=['date', 'geoId','cases_per_100khab', 'deaths_per_100khab',
       'first_dose_100khab', 'second_dose_100khab']

In [68]:
data_ready=data_ready[new_columns]
data_ready.head()

,date,geoId,cases_per_100khab,deaths_per_100khab,first_dose_100khab,second_dose_100khab
0,2020-01-04,ES,0.002113,0.0,0.0,0.0
1,2020-01-04,FI,0.018099,0.0,0.0,0.0
2,2020-01-04,CZ,0.000000,0.0,0.0,0.0
3,2020-01-04,DE,0.001202,0.0,0.0,0.0
4,2020-01-04,ES,0.033803,0.0,0.0,0.0


In [69]:
data_ready['cmltve_cases']=data_ready.groupby(['geoId'])['cases_per_100khab'].cumsum()
data_ready['cmltve_deaths']=data_ready.groupby(['geoId'])['deaths_per_100khab'].cumsum()
data_ready['cmltve_first_dose']=data_ready.groupby(['geoId'])['first_dose_100khab'].cumsum()
data_ready['cmltve_second_dose']=data_ready.groupby(['geoId'])['second_dose_100khab'].cumsum()

<ipython-input-69-adfb01820e4d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ready['cmltve_cases']=data_ready.groupby(['geoId'])['cases_per_100khab'].cumsum()
<ipython-input-69-adfb01820e4d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ready['cmltve_deaths']=data_ready.groupby(['geoId'])['deaths_per_100khab'].cumsum()
<ipython-input-69-adfb01820e4d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [72]:
data_ready[data_ready['geoId']=='ES'].head(10)

,date,geoId,cases_per_100khab,deaths_per_100khab,first_dose_100khab,second_dose_100khab,cmltve_cases,cmltve_deaths,cmltve_first_dose,cmltve_second_dose
0,2020-01-04,ES,0.002113,0.000000,0.0,0.0,0.002113,0.000000,0.0,0.0
4,2020-01-04,ES,0.033803,0.000000,0.0,0.0,0.035916,0.000000,0.0,0.0
15,2020-01-04,ES,1.445092,0.000000,0.0,0.0,1.481008,0.000000,0.0,0.0
38,2020-01-04,ES,11.499471,1.890874,0.0,0.0,12.980479,1.890874,0.0,0.0
68,2020-01-04,ES,1.527488,0.371837,0.0,0.0,14.507967,2.262710,0.0,0.0
98,2020-01-11,ES,0.836632,0.073945,0.0,0.0,15.344599,2.336655,0.0,0.0
128,2020-01-11,ES,1.174666,0.021127,0.0,0.0,16.519265,2.357782,0.0,0.0
158,2020-01-11,ES,7.612088,0.025352,0.0,0.0,24.131353,2.383135,0.0,0.0
188,2020-01-11,ES,24.133465,0.143664,0.0,0.0,48.264818,2.526799,0.0,0.0
218,2020-01-11,ES,22.303860,0.285216,0.0,0.0,70.568678,2.812015,0.0,0.0


In [71]:
#data_ready.to_csv('../datasets_Tableau_clean/data_ready.csv')

### Conclusions regarding the data
Merging the cases/deaths datasets and the vaccination dataset this way will not work. The problem is that the new_cases dataset reports on a daily basis, whereas the vaccination dataset works on a weekly basis. For that reason, they have to be dealt with separately.

For clarity, we will do it on a different notebook.